In [1]:
import pandas as pd 
import dask as dd 
import numpy as np

In [2]:

df_principals = pd.read_csv("../data/title.principals.tsv" , sep = "\t") 

In [3]:
df_principals.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Herself""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N


In [4]:
df_principals = df_principals[ ["tconst" , "nconst"]] # id_pelicula , Id actor]

In [5]:
df_principals.columns = [ "id_movie" , "id_actor"] 

In [7]:
df_name = pd.read_csv("../data/name.basics.tsv" , sep = "\t") 

In [8]:
df_name = df_name[["nconst" , "primaryName"] ]
df_name.columns = [ "id_actor" , "name"]

In [9]:
df_name.head()

,id_actor,name
0,nm0000001,Fred Astaire
1,nm0000002,Lauren Bacall
2,nm0000003,Brigitte Bardot
3,nm0000004,John Belushi
4,nm0000005,Ingmar Bergman


In [10]:
df_merge = df_principals.merge( df_name , on = "id_actor" )

In [14]:
gp = df_merge[:1000].groupby( by = [ "id_movie"] )["name"].agg( [ lambda x : ",".join( [  name.replace(" " , "") for name in x ]  ) ])

In [16]:
df_merge.shape

(35498702, 3)

In [17]:
df_merge.head()

,id_movie,id_actor,name
0,tt0000001,nm1588970,Carmencita
1,tt7513040,nm1588970,Carmencita
2,tt0000001,nm0005690,William K.L. Dickson
3,tt0000005,nm0005690,William K.L. Dickson
4,tt0000006,nm0005690,William K.L. Dickson


In [19]:
df_wiki = pd.read_csv("../data/wiki_movie_plots_deduped.csv")

In [20]:
df_wiki.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


# Code

In [48]:
df_title = pd.read_csv("../data/title.basics.tsv" , sep = "\t")

/home/afhuertas/data-science/env-sonnet/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [50]:
df_rat = pd.read_csv( "../data/title.ratings.tsv" , sep = "\t" )
df_rat.head()

,tconst,averageRating,numVotes
0,tt0000001,5.6,1535
1,tt0000002,6.1,185
2,tt0000003,6.5,1186
3,tt0000004,6.2,113
4,tt0000005,6.1,1903


In [30]:
df_title.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [31]:
df_title = df_title[ [ "tconst" , "originalTitle" , "startYear"] ]
df_title.columns = [ "id_movie" , "title" , "year"] 

In [32]:
df_title.head()

,id_movie,title,year
0,tt0000001,Carmencita,1894
1,tt0000002,Le clown et ses chiens,1892
2,tt0000003,Pauvre Pierrot,1892
3,tt0000004,Un bon bock,1892
4,tt0000005,Blacksmith Scene,1893


In [33]:
df_wiki = pd.read_csv( "../data/wiki_movie_plots_deduped.csv" )

In [34]:
df_wiki.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [35]:
df_wiki = df_wiki[ ["Title" , "Plot" , "Release Year"]]
df_wiki.columns = ["title" , "plot" , "year"]

In [44]:
df_title["year"] = df_title["year"].astype( str )
df_wiki["year"] = df_wiki["year"].astype( str )

In [45]:
df_merge2 = df_title.merge( df_wiki , on =[ "title" , "year" ], how = "inner" )

In [46]:
df_merge2.head()

,id_movie,title,year,plot
0,tt0000399,Jack and the Beanstalk,1902,The earliest known adaptation of the classic f...
1,tt0000420,Alice in Wonderland,1903,"Alice follows a large white rabbit down a ""Rab..."
2,tt0000439,The Great Train Robbery,1903,The film opens with two bandits breaking into ...
3,tt0000487,The Great Train Robbery,1903,The film opens with two bandits breaking into ...
4,tt0000488,The Land Beyond the Sunset,1912,Joe is an impoverished New York newsboy who li...


In [47]:
df_merge2.tail()

,id_movie,title,year,plot
34283,tt9399328,Ninnu Kalisaka,2009,The story is a tale of two young pairs of love...
34284,tt9695066,Cuckoo,2002,"September 1944. Several days before Finland, a..."
34285,tt9815072,Pontianak Menjerit,2005,Datuk Pengiran Abdul Rahman has asked his lawy...
34286,tt9829966,Samhita,2013,Revati Sathe is a National Award winning docum...
34287,tt9881364,Gaja,2008,Gaja (Darshan) and Krishna are friends. Gaja g...


In [49]:
df_merge2.shape 

(34288, 4)

In [53]:
df_rat = df_rat[ ["tconst" , "averageRating"]]  # .head()
df_rat.columns = ["id_movie" , "ratings"]
df_rat.head()

,id_movie,ratings
0,tt0000001,5.6
1,tt0000002,6.1
2,tt0000003,6.5
3,tt0000004,6.2
4,tt0000005,6.1


In [55]:
df_final = df_merge2.merge( df_rat , on = "id_movie" , how = "left" )

In [56]:
df_final.head()

,id_movie,title,year,plot,ratings
0,tt0000399,Jack and the Beanstalk,1902,The earliest known adaptation of the classic f...,6.1
1,tt0000420,Alice in Wonderland,1903,"Alice follows a large white rabbit down a ""Rab...",6.3
2,tt0000439,The Great Train Robbery,1903,The film opens with two bandits breaking into ...,7.3
3,tt0000487,The Great Train Robbery,1903,The film opens with two bandits breaking into ...,7.1
4,tt0000488,The Land Beyond the Sunset,1912,Joe is an impoverished New York newsboy who li...,6.9
